In [75]:
import pandas as pd
from hnmchallenge.data_reader import DataReader
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from hnmchallenge.dataset import Dataset
from hnmchallenge.stratified_dataset import StratifiedDataset
from hnmchallenge.filtered_dataset import FilterdDataset
from hnmchallenge.models.top_pop import TopPop
from hnmchallenge.evaluation.python_evaluation import map_at_k
from hnmchallenge.constant import *

In [76]:
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [77]:
dataset = FilterdDataset()
dr = DataReader()

In [78]:
fd = dr.get_filtered_full_data()

In [62]:
fd

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,0,0,0.008458,2
1,2018-09-20,0,0,0.008458,2
2,2018-09-20,1,1,0.033881,1
3,2018-09-20,2,2,0.021593,1
4,2018-09-20,3,3,0.022017,2
...,...,...,...,...,...
14361083,2020-09-22,198005,20308,0.059305,2
14361084,2020-09-22,198005,16094,0.042356,2
14361085,2020-09-22,798833,21707,0.043203,1
14361086,2020-09-22,395317,4994,0.006763,1


In [79]:
df=fd.drop_duplicates(["t_dat",DEFAULT_USER_COL])

In [80]:
df

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,0,0,0.008458,2
2,2018-09-20,1,1,0.033881,1
3,2018-09-20,2,2,0.021593,1
4,2018-09-20,3,3,0.022017,2
6,2018-09-20,4,4,0.032186,2
...,...,...,...,...,...
14361074,2020-09-22,59779,20150,0.050831,2
14361083,2020-09-22,198005,20308,0.059305,2
14361085,2020-09-22,798833,21707,0.043203,1
14361086,2020-09-22,395317,4994,0.006763,1


In [81]:
df['date_diff'] = (df.pop('t_dat').groupby(df['customer_id']).diff().dt.days.fillna(0, downcast='infer'))

/tmp/ipykernel_23885/1392639203.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date_diff'] = (df.pop('t_dat').groupby(df['customer_id']).diff().dt.days.fillna(0, downcast='infer'))


In [82]:
df

,customer_id,article_id,price,sales_channel_id,date_diff
0,0,0,0.008458,2,0
2,1,1,0.033881,1,0
3,2,2,0.021593,1,0
4,3,3,0.022017,2,0
6,4,4,0.032186,2,0
...,...,...,...,...,...
14361074,59779,20150,0.050831,2,17
14361083,198005,20308,0.059305,2,2
14361085,798833,21707,0.043203,1,21
14361086,395317,4994,0.006763,1,11


In [85]:
df[df[DEFAULT_USER_COL]==0]

,customer_id,article_id,price,sales_channel_id,date_diff
0,0,0,0.008458,2,0
3102169,0,0,0.007610,2,319
3265030,0,644,0.039576,2,11
4089565,0,1952,0.030492,2,51
8123440,0,1482,0.033881,2,192


In [83]:
user_average=df.groupby(DEFAULT_USER_COL)["date_diff"].mean().reset_index()

In [84]:
user_average

,customer_id,date_diff
0,0,114.600000
1,1,53.230769
2,2,67.125000
3,3,124.333333
4,4,0.000000
...,...,...
1136201,1136201,0.000000
1136202,1136202,0.000000
1136203,1136203,0.000000
1136204,1136204,0.000000


In [63]:
count_mb=fd.groupby(DEFAULT_USER_COL)["sales_channel_id"].size().reset_index()

In [64]:
count_mb

,customer_id,sales_channel_id
0,0,14
1,1,24
2,2,26
3,3,7
4,4,1
...,...,...
1136201,1136201,2
1136202,1136202,1
1136203,1136203,3
1136204,1136204,2


In [65]:
df=fd[fd["sales_channel_id"]==1]

In [66]:
count_mb1=df.groupby(DEFAULT_USER_COL)["sales_channel_id"].size().reset_index()

In [70]:
df.groupby(DEFAULT_USER_COL)["sales_channel_id"].size()

customer_id
1           7
2          26
5           2
6          18
7           9
           ..
1136190     6
1136194     7
1136196     1
1136197     2
1136202     1
Name: sales_channel_id, Length: 610744, dtype: int64

In [67]:
count=pd.merge(count_mb,count_mb1, on =DEFAULT_USER_COL, how='left')

In [68]:
count

,customer_id,sales_channel_id_x,sales_channel_id_y
0,0,14,NaN
1,1,24,7.0
2,2,26,26.0
3,3,7,NaN
4,4,1,NaN
...,...,...,...
1136201,1136201,2,NaN
1136202,1136202,1,1.0
1136203,1136203,3,NaN
1136204,1136204,2,NaN


In [74]:
count=count.fillna(0)

,customer_id,sales_channel_id_x,sales_channel_id_y,sales_channel_score
0,0,14,0.0,0.000000
1,1,24,7.0,0.291667
2,2,26,26.0,1.000000
3,3,7,0.0,0.000000
4,4,1,0.0,0.000000
...,...,...,...,...
1136201,1136201,2,0.0,0.000000
1136202,1136202,1,1.0,1.000000
1136203,1136203,3,0.0,0.000000
1136204,1136204,2,0.0,0.000000


In [72]:
count["sales_channel_score"]=count["sales_channel_id_y"]/count["sales_channel_id_x"]

In [73]:
count

,customer_id,sales_channel_id_x,sales_channel_id_y,sales_channel_score
0,0,14,NaN,NaN
1,1,24,7.0,0.291667
2,2,26,26.0,1.000000
3,3,7,NaN,NaN
4,4,1,NaN,NaN
...,...,...,...,...
1136201,1136201,2,NaN,NaN
1136202,1136202,1,1.0,1.000000
1136203,1136203,3,NaN,NaN
1136204,1136204,2,NaN,NaN


In [40]:
count_mb1

customer_id
1           7
2          26
5           2
6          18
7           9
           ..
1136190     6
1136194     7
1136196     1
1136197     2
1136202     1
Name: sales_channel_id, Length: 610744, dtype: int64

In [ ]:
fd[DEFAULT_USER_COL].nunique()

In [ ]:
fd = fd.drop_duplicates([DEFAULT_USER_COL, DEFAULT_ITEM_COL])

In [ ]:
fd

In [ ]:
count_mb = fd.groupby(DEFAULT_ITEM_COL).count()
feature = count_mb.reset_index()[[DEFAULT_ITEM_COL, "t_dat"]].rename(
            columns={"t_dat": "popularity"})

In [ ]:
feature

In [ ]:
feature["popularity_score"]=(feature["popularity"]-feature["popularity"].min())/(feature["popularity"].max()-feature["popularity"].min())

In [ ]:
feature

In [ ]:
feature['rank'] = feature['popularity_score'].rank(ascending=False,method='min').astype(int)

In [ ]:
feature

In [ ]:
df=dr.get_filtered_full_data()

In [ ]:
df= df.drop_duplicates([DEFAULT_USER_COL, DEFAULT_ITEM_COL])

In [ ]:
df

In [ ]:
df["last_buy"] = df.groupby(DEFAULT_USER_COL)["t_dat"].transform(max)
df["first_buy"] = df.groupby(DEFAULT_USER_COL)["t_dat"].transform(min)
df["time_score"] = (df["t_dat"] - df["first_buy"]) / (
            df["last_buy"] - df["first_buy"]
        )

In [ ]:
df=df.fillna(1)#df["tdiff"] =1/ ((df["last_buy"]-df["t_dat"]).dt.days) 

In [ ]:
df

In [ ]:
df["rank_time"] = df.groupby(DEFAULT_USER_COL)["time_score"].rank(ascending=False,method='min').astype(int)

In [ ]:
df

In [ ]:
df.loc[df[DEFAULT_USER_COL]==1]

In [ ]:
feature

In [ ]:
final=pd.merge(df, feature, on =DEFAULT_ITEM_COL, how='left')

In [ ]:
final

In [ ]:
final.loc[final[DEFAULT_USER_COL]==1]

In [ ]:
user=dr.get_filtered_full_data()

In [ ]:
user = user.drop_duplicates([DEFAULT_USER_COL])

In [ ]:
user['temp']=1

In [ ]:
user=user[[DEFAULT_USER_COL,'temp']]

In [ ]:
user

In [ ]:
feature_100=feature[feature['rank']<=100]

In [ ]:
feature_100['temp']=1

In [ ]:
final1=pd.merge(user, feature_100, on='temp')

In [ ]:
final1

In [ ]:
final1=final1.drop('temp', axis=1)

In [ ]:
final1

In [ ]:
final1[final1['customer_id']==1]

In [ ]:
final

In [ ]:
final=final.drop(['popularity_score','rank','popularity','t_dat','price','sales_channel_id','last_buy','first_buy'], axis=1)

In [ ]:
final2=pd.merge(final,final1,on=[DEFAULT_USER_COL,DEFAULT_ITEM_COL],how='outer')

In [ ]:
final[final[DEFAULT_USER_COL]==1]

In [ ]:
final2[final2[DEFAULT_USER_COL]==1]

In [ ]:
mask = final2.isnull().any(axis=1)

In [ ]:
final3=final2[~mask]

In [ ]:
final3[final3[DEFAULT_USER_COL]==1]

In [ ]:
final2=final2.fillna(0)

In [ ]:
final2

In [ ]:
final2=final2.drop(['rank_time','rank'],axis=1)

In [ ]:
a=0.95
eps=1e-6
final2['score']=a*final2['time_score']+(1-a+eps)*final2['popularity_score']

In [ ]:
final2

In [ ]:
final2["rank"] = final2.groupby(DEFAULT_USER_COL)["score"].rank(ascending=False,method='min').astype(int)

In [ ]:
final2

In [ ]:
final2=final2.drop(["time_score","popularity","popularity_score"],axis=1)

In [ ]:
final2[DEFAULT_USER_COL].nunique()

In [ ]:
final2

In [ ]:
cutoff = final2.groupby(DEFAULT_USER_COL).size().values

In [ ]:
cutoff

In [ ]:
i=0
filter_indices = []
for cut in cutoff:
    filter_indices.extend(range(i, i+12))
    i=i+cut

In [ ]:
filter_indices

In [ ]:
final2.head()

In [ ]:
final2 = final2.sort_values(DEFAULT_USER_COL, ascending=True)

In [ ]:
final2.head()

In [ ]:
final2 = final2.sort_values([DEFAULT_USER_COL, "rank"], ascending=True)

In [ ]:
final2

In [ ]:
final2 = final2.reset_index(drop=True)

In [ ]:
final_df = final2.loc[filter_indices]

In [ ]:
final_df[DEFAULT_USER_COL].nunique()

In [ ]:
final_final_df = final_df.drop(["score","rank"] ,axis=1)

In [ ]:
final_final_df[DEFAULT_USER_COL].nunique()

In [ ]:
from hnmchallenge.submission_handler import SubmissionHandler
sh = SubmissionHandler()     

In [ ]:
sh.create_submission_filtered_data([final_final_df], sub_name="Time_Pop")